In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data_processed.csv')
df = df[['Article Citation Count', 'Author Countries']]
df['Author Countries'] = df['Author Countries'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
df = df[~df['Author Countries'].isna()]
df = df.fillna(0)
df

,Article Citation Count,Author Countries
0,16.0,"[Canada, Canada, Canada]"
1,28.0,"[Republic of Singapore, Commonwealth of Austra..."
2,50.0,"[Canada, Canada, Canada]"
3,15.0,"[People's Republic of China, People's Republic..."
4,16.0,"[Islamic Republic of Iran, Islamic Republic of..."
...,...,...
41494,0.0,"[Islamic Republic of Iran, Islamic Republic of..."
41495,0.0,"[Islamic Republic of Iran, Islamic Republic of..."
41496,0.0,"[Italy, Italy]"
41497,0.0,"[Republic of Korea, Republic of Korea, Republi..."


In [3]:
df['Collabs'] = df['Author Countries']
df.columns = ['Cites', 'Base', 'Outgoing']

In [4]:
df = df.explode('Base')
df

,Cites,Base,Outgoing
0,16.0,Canada,"[Canada, Canada, Canada]"
0,16.0,Canada,"[Canada, Canada, Canada]"
0,16.0,Canada,"[Canada, Canada, Canada]"
1,28.0,Republic of Singapore,"[Republic of Singapore, Commonwealth of Austra..."
1,28.0,Commonwealth of Australia,"[Republic of Singapore, Commonwealth of Austra..."
...,...,...,...
41498,3.0,People's Republic of China,"[People's Republic of China, People's Republic..."
41498,3.0,People's Republic of China,"[People's Republic of China, People's Republic..."
41498,3.0,People's Republic of China,"[People's Republic of China, People's Republic..."
41498,3.0,People's Republic of China,"[People's Republic of China, People's Republic..."


In [5]:
def remove_one_string_from_list(string, lst):
    new_list = lst.copy()  
    new_list.remove(string)  
    return new_list


In [6]:
df['Outgoing'] = df.apply(lambda row: remove_one_string_from_list(row.Base, row['Outgoing']), axis=1)
df

,Cites,Base,Outgoing
0,16.0,Canada,"[Canada, Canada]"
0,16.0,Canada,"[Canada, Canada]"
0,16.0,Canada,"[Canada, Canada]"
1,28.0,Republic of Singapore,"[Commonwealth of Australia, Republic of Singap..."
1,28.0,Commonwealth of Australia,"[Republic of Singapore, Republic of Singapore,..."
...,...,...,...
41498,3.0,People's Republic of China,"[People's Republic of China, People's Republic..."
41498,3.0,People's Republic of China,"[People's Republic of China, People's Republic..."
41498,3.0,People's Republic of China,"[People's Republic of China, People's Republic..."
41498,3.0,People's Republic of China,"[People's Republic of China, People's Republic..."


In [9]:
df = df.explode('Outgoing')
df = df.reset_index().drop_duplicates()
df

,level_0,index,Cites,Base,Outgoing
0,0,0,16.0,Canada,Canada
1,6,1,28.0,Republic of Singapore,Commonwealth of Australia
2,7,1,28.0,Republic of Singapore,Republic of Singapore
3,9,1,28.0,Commonwealth of Australia,Republic of Singapore
4,11,1,28.0,Commonwealth of Australia,Commonwealth of Australia
...,...,...,...,...,...
73903,1278871,41494,0.0,Islamic Republic of Iran,Islamic Republic of Iran
73904,1278873,41495,0.0,Islamic Republic of Iran,Islamic Republic of Iran
73905,1278875,41496,0.0,Italy,Italy
73906,1278877,41497,0.0,Republic of Korea,Republic of Korea


In [10]:
df.drop(columns='level_0', axis=1, inplace=True)
df

,index,Cites,Base,Outgoing
0,0,16.0,Canada,Canada
1,1,28.0,Republic of Singapore,Commonwealth of Australia
2,1,28.0,Republic of Singapore,Republic of Singapore
3,1,28.0,Commonwealth of Australia,Republic of Singapore
4,1,28.0,Commonwealth of Australia,Commonwealth of Australia
...,...,...,...,...
73903,41494,0.0,Islamic Republic of Iran,Islamic Republic of Iran
73904,41495,0.0,Islamic Republic of Iran,Islamic Republic of Iran
73905,41496,0.0,Italy,Italy
73906,41497,0.0,Republic of Korea,Republic of Korea


In [11]:
df = df[df.Base != '']
df = df[df.Outgoing != '']
df

,index,Cites,Base,Outgoing
0,0,16.0,Canada,Canada
1,1,28.0,Republic of Singapore,Commonwealth of Australia
2,1,28.0,Republic of Singapore,Republic of Singapore
3,1,28.0,Commonwealth of Australia,Republic of Singapore
4,1,28.0,Commonwealth of Australia,Commonwealth of Australia
...,...,...,...,...
73903,41494,0.0,Islamic Republic of Iran,Islamic Republic of Iran
73904,41495,0.0,Islamic Republic of Iran,Islamic Republic of Iran
73905,41496,0.0,Italy,Italy
73906,41497,0.0,Republic of Korea,Republic of Korea


In [22]:
collabs = df.groupby(['Base', 'Outgoing']).Cites.sum()
collabs

Base                      Outgoing                
Africa                    Africa                           2.0
                          Republic of South Africa         2.0
Arab Republic of Egypt    Arab Republic of Egypt        1321.0
                          Canada                          48.0
                          Czech Republic                  26.0
                                                        ...   
United States of America  Ukraine                        220.0
                          United Arab Emirates           550.0
                          United Kingdom                9624.0
                          United Mexican States          659.0
                          United States of America    343559.0
Name: Cites, Length: 2370, dtype: float64

In [23]:
data = pd.DataFrame(collabs)
data.reset_index(inplace=True)
data

,Base,Outgoing,Cites
0,Africa,Africa,2.0
1,Africa,Republic of South Africa,2.0
2,Arab Republic of Egypt,Arab Republic of Egypt,1321.0
3,Arab Republic of Egypt,Canada,48.0
4,Arab Republic of Egypt,Czech Republic,26.0
...,...,...,...
2365,United States of America,Ukraine,220.0
2366,United States of America,United Arab Emirates,550.0
2367,United States of America,United Kingdom,9624.0
2368,United States of America,United Mexican States,659.0


In [25]:
mask = data['Base'] == data['Outgoing']

# Filter the DataFrame using the mask to drop rows with the same values in column1 and column2
df_filtered = data[~mask]
df_filtered        

,Base,Outgoing,Cites
1,Africa,Republic of South Africa,2.0
3,Arab Republic of Egypt,Canada,48.0
4,Arab Republic of Egypt,Czech Republic,26.0
5,Arab Republic of Egypt,France,60.0
6,Arab Republic of Egypt,Germany,14.0
...,...,...,...
2364,United States of America,Taiwan,9314.0
2365,United States of America,Ukraine,220.0
2366,United States of America,United Arab Emirates,550.0
2367,United States of America,United Kingdom,9624.0


In [24]:
df_filtered.to_csv('collabs_num.csv', index=False)